# NZ Seek Data Analysis

## Data preprocessing

### Import Libs

In [ ]:
import pandas as pd
import numpy as np
import datetime

### Main Functions

In [ ]:
df_job_name = pd.read_excel("path", header = None)

clean_Posted_Time(df, DICT_DATE_INDEX[Job_Name])

clean_Classification(df, DICT_CLASSFICATION_INDEX[Job_Name])

clean_Salary(df, DICT_SALARY_FLOOR[Job_Name], DICT_SALARY_CEILING[Job_Name])
    
clean_Redundancies(df, Job_Name)

### Constant values

In [ ]:
DICT_AREA_INDEX = {'Admin':3}
DICT_DATE_INDEX = {'Admin':4}
DICT_CLASSFICATION_INDEX = {'Admin':5}
DICT_SALARY_FLOOR = {'Admin':35000}
DICT_SALARY_CEILING = {'Admin':55000}
HOURLY_SALARY_TO_ANNUAL = 8*200
CORRECTING_WRONG_ANNUAL_SALARY_FORMAT = *1000

### Useful Functions

In [ ]:
def deduplication(x):
    mid = len(x)//2
    if x!='unknown':
        return x[:mid+1]
    else:
        return 'unknown'
    
def apply_salary(x):
    if 'classification:' in x:
        return 'unknown'
    else:
        return x
    
def apply_classification(x):
    if 'classification' in x:
        return x[15:]
    else:
        return 'unknown'
    
def apply_posted_time(x):
    if 'd' in x:
        return -int(x[:-1])
    if 'm' in x:
        return -(int(x[:-1])*30)
    else:
        return 0

### Cleaning Locations and Areas

In [ ]:
df_admin[['Location', 'Area']] = df_admin[Area_Index].str.split('area:', expand=True) 

df_admin['Location'] = df_admin['Location'].apply(lambda x: x[Location_Index:]) 

df_admin['Location'] = df_admin['Location'].apply(deduplication)

df_admin['Area'].fillna(value='unknown',inplace = True) 

df_admin['Area1'] = df_admin['Area1'].apply(deduplication)

### Clean Classification

In [ ]:
df_admin[['Subclassification','Classification']] = df_admin[Classification_Index].str.split('subClassification:', expand = True)

df_admin['Classification'].fillna(value='unknown',inplace = True) 

df_admin['Classification'] = df_admin['Classification'].apply(deduplication)

df_admin['Salary'] = df_admin['Subclassification'].apply(apply_salary)

df_admin['Subclassification']=df_admin['Subclassification'].apply(apply_classification)

df_admin[['Class1','Class2','Class3']] = df_admin['Subclassification'].str.split('&', expand = True).fillna('unknown')

### Clean Posted Time

In [ ]:
df_admin[['Posted time','Featured at']] = df_admin[Posted_Time_Index].str.split(',', expand = True)[[0,2]]

df_admin['Featured at'].fillna('unknown', inplace = True)

df_admin['Posted time'] = df_admin['Posted time'].str.extract('(\d+[a-z])') 

df_admin['Posted time'].fillna('unknown', inplace = True)

df_admin['Posted time'] = df_admin['Posted time'].astype('string')

df_admin['Posted time']=df_admin['Posted time'].apply(apply_posted_time)

### Clean Salary

In [ ]:
df_admin['Salary'] = df_admin[df_admin['Salary'].str.contains('\d', na=False)]['Salary'].str.replace('to','-',regex=True)

df_admin['Salary'] = df_admin[df_admin['Salary'].str.contains('\d', na=False)]['Salary'].str.replace('\d\%','',regex=True)

df_admin['Salary'] = df_admin[df_admin['Salary'].str.contains('\d', na=False)]['Salary'].str.replace(',','',regex=True)

df_admin['Salary'].fillna('unknown')

df_admin[['Lo_Salary','Hi_Salary']] = df_admin['Salary'].str.split('-', n=1, expand = True).fillna('unknown')

df_admin['Lo_Salary'] = df_admin['Lo_Salary'].str.extract('(\d+\.\d+|\d+\d+|\d+k|\d+)')
df_admin['Hi_Salary'] = df_admin['Hi_Salary'].str.extract('(\d+\.\d+|\d+\d+|\d+k|\d+)')

df_admin['Lo_Salary'] = df_admin[df_admin['Lo_Salary'].str.contains('\d', na=False)]['Lo_Salary'].str.replace('k', '000', regex=True)

df_admin['Hi_Salary'] = df_admin[df_admin['Hi_Salary'].str.contains('\d', na=False)]['Hi_Salary'].str.replace('k', '000', regex=True)

df_admin['Lo_Salary'] = df_admin[df_admin['Lo_Salary'].str.contains('\d', na=False)]['Lo_Salary'].str.replace(' ', '', regex=True)

df_admin['Hi_Salary'] = df_admin[df_admin['Hi_Salary'].str.contains('\d', na=False)]['Hi_Salary'].str.replace(' ', '', regex=True)

df_admin['Lo_Salary'] = df_admin['Lo_Salary'].fillna("unknown")
df_admin['Hi_Salary'] = df_admin['Hi_Salary'].fillna("unknown")

df_admin.loc[df_admin['Lo_Salary'] == 'unknown', 'Lo_Salary'] = df_admin['Lo_Salary']
df_admin.loc[df_admin['Hi_Salary'] == 'unknown', 'Hi_Salary'] = df_admin['Hi_Salary']


# Setting random salary

for r, row in enumerate(df_admin['Lo_Salary'].values):
    if row == 'unknown':
        df_admin['Lo_Salary'][r] = np.random.randint(Salary_Floor, Salary_Ceiling, size = 1)[0]
        
for r, row in enumerate(df_admin['Hi_Salary'].values):
    if row == 'unknown':
        df_admin['Hi_Salary'][r] = max(np.random.randint(Salary_Floor, Salary_Ceiling, size = 1)[0], df_admin['Lo_Salary'][r] )

df_admin['Lo_Salary'] = df_admin['Lo_Salary'].astype('float32')
df_admin['Hi_Salary'] = df_admin['Hi_Salary'].astype('float32')


# Transforming the hourly paid to the annual salary

df_admin.loc[df_admin['Lo_Salary'] < 50, 'Lo_Salary'] = df_admin['Lo_Salary']*HOURLY_SALARY_TO_ANNUAL
df_admin.loc[df_admin['Hi_Salary'] < 50, 'Hi_Salary'] = df_admin['Hi_Salary']*HOURLY_SALARY_TO_ANNUAL


# Correcting the wrong format of the annual salary

df_admin.loc[(df_admin['Lo_Salary'] >= 60) & (df_admin['Lo_Salary'] < 90), 'Lo_Salary'] = df_admin['Lo_Salary']*CORRECTING_WRONG_ANNUAL_SALARY_FORMAT
df_admin.loc[(df_admin['Hi_Salary'] >= 60) & (df_admin['Hi_Salary'] < 90), 'Hi_Salary'] = df_admin['Hi_Salary']*CORRECTING_WRONG_ANNUAL_SALARY_FORMAT

### Clean redundancies

In [ ]:
# Drop unuseful columns

df_admin.drop(3, axis = 1, inplace = True) 
df_admin.drop('Area', axis = 1, inplace = True) 
df_admin.drop('Area2', axis = 1, inplace = True) 
df_admin.drop(5, axis = 1, inplace = True)
df_admin.drop(4, axis = 1, inplace = True)
df_admin.drop('Subclassification', axis = 1, inplace = True)
df_admin.drop(['Featured at'], axis = 1, inplace = True)


# Rename & Fillna

df_admin.rename(columns ={'Area1': 'Area'}, inplace = True )
df_admin.rename(columns ={0:'Job title',1:'Web add',2:'Co name'}, inplace = True )
df_admin['Co name'].fillna(df_admin['Featured at'], inplace = True)